**Graph 1:**

Unblockable backdoor path. Frontdoor path cannot be blocked either: unable to use rule 1 or rule 2 ($y \not\perp z_1$, and there are no variables on which we can condition to make the two independent).

**Graph 2:**